In [1]:
from index import BS4Parser, Document, Image, CorpusClient
BS4CLIENT = BS4Parser()

In [2]:
model = BS4CLIENT.process_local_file("data/BULLETINS/67068.htm")
model.images = [Image(url="lol", legende="aha"), Image(url="OmO", legende="UwU")]
model

Document(fichier='67068.htm', numero='258', date=datetime.datetime(2011, 6, 21, 0, 0), rubrique='Focus', titre='Physique : Mathias Fink, un bel exemple de chercheur qui innove', auteur='ADIT - Jean-François Desessard - email : jfd@adit.fr', contact='Institut Langevin "Ondes et Images" - Mathias Fink - email : mathias.fink@espci.fr - http://www.institut-langevin.espci.fr', texte='Le 27 avril dernier, le CNRS décernait pour la première fois la Médaille de l\'Innovation, dont Valérie Pécresse, ministre de l\'Enseignement Supérieur et de la Recherche est à l\'origine de la création. Le CNRS souhaite ainsi honorer des chercheurs et ingénieurs travaillant dans des établissements publics de recherche, des universités, des grandes écoles mais aussi des industriels qui développent des innovations marquantes. Pour cette première édition, cette nouvelle distinction a été attribuée à trois chercheurs réputés : l\'économiste Esther Duflo, le roboticien François Pierrot et le physicien Mathias Fink.

In [3]:
import datetime
model.date = datetime.datetime.now()
model

Document(fichier='67068.htm', numero='258', date=datetime.datetime(2025, 3, 20, 15, 40, 33, 976067), rubrique='Focus', titre='Physique : Mathias Fink, un bel exemple de chercheur qui innove', auteur='ADIT - Jean-François Desessard - email : jfd@adit.fr', contact='Institut Langevin "Ondes et Images" - Mathias Fink - email : mathias.fink@espci.fr - http://www.institut-langevin.espci.fr', texte='Le 27 avril dernier, le CNRS décernait pour la première fois la Médaille de l\'Innovation, dont Valérie Pécresse, ministre de l\'Enseignement Supérieur et de la Recherche est à l\'origine de la création. Le CNRS souhaite ainsi honorer des chercheurs et ingénieurs travaillant dans des établissements publics de recherche, des universités, des grandes écoles mais aussi des industriels qui développent des innovations marquantes. Pour cette première édition, cette nouvelle distinction a été attribuée à trois chercheurs réputés : l\'économiste Esther Duflo, le roboticien François Pierrot et le physicien

In [4]:
xml = model.model_dump_xml_str()
xml

'<Document><fichier>67068.htm</fichier><numero>258</numero><date>2025-03-20T15:40:33.976067</date><rubrique>Focus</rubrique><titre>Physique : Mathias Fink, un bel exemple de chercheur qui innove</titre><auteur>ADIT - Jean-François Desessard - email : jfd@adit.fr</auteur><contact>Institut Langevin "Ondes et Images" - Mathias Fink - email : mathias.fink@espci.fr - http://www.institut-langevin.espci.fr</contact><texte>Le 27 avril dernier, le CNRS décernait pour la première fois la Médaille de l\'Innovation, dont Valérie Pécresse, ministre de l\'Enseignement Supérieur et de la Recherche est à l\'origine de la création. Le CNRS souhaite ainsi honorer des chercheurs et ingénieurs travaillant dans des établissements publics de recherche, des universités, des grandes écoles mais aussi des industriels qui développent des innovations marquantes. Pour cette première édition, cette nouvelle distinction a été attribuée à trois chercheurs réputés : l\'économiste Esther Duflo, le roboticien François 

In [5]:
Document.model_validate_xml(xml)

Document(fichier='67068.htm', numero='258', date=datetime.datetime(2025, 3, 20, 15, 40, 33, 976067), rubrique='Focus', titre='Physique : Mathias Fink, un bel exemple de chercheur qui innove', auteur='ADIT - Jean-François Desessard - email : jfd@adit.fr', contact='Institut Langevin "Ondes et Images" - Mathias Fink - email : mathias.fink@espci.fr - http://www.institut-langevin.espci.fr', texte='Le 27 avril dernier, le CNRS décernait pour la première fois la Médaille de l\'Innovation, dont Valérie Pécresse, ministre de l\'Enseignement Supérieur et de la Recherche est à l\'origine de la création. Le CNRS souhaite ainsi honorer des chercheurs et ingénieurs travaillant dans des établissements publics de recherche, des universités, des grandes écoles mais aussi des industriels qui développent des innovations marquantes. Pour cette première édition, cette nouvelle distinction a été attribuée à trois chercheurs réputés : l\'économiste Esther Duflo, le roboticien François Pierrot et le physicien

In [6]:
BS4CLIENT.process_folder("data/BULLETINS", limit=5)

[Document(fichier='67068.htm', numero='258', date=datetime.datetime(2011, 6, 21, 0, 0), rubrique='Focus', titre='Physique : Mathias Fink, un bel exemple de chercheur qui innove', auteur='ADIT - Jean-François Desessard - email : jfd@adit.fr', contact='Institut Langevin "Ondes et Images" - Mathias Fink - email : mathias.fink@espci.fr - http://www.institut-langevin.espci.fr', texte='Le 27 avril dernier, le CNRS décernait pour la première fois la Médaille de l\'Innovation, dont Valérie Pécresse, ministre de l\'Enseignement Supérieur et de la Recherche est à l\'origine de la création. Le CNRS souhaite ainsi honorer des chercheurs et ingénieurs travaillant dans des établissements publics de recherche, des universités, des grandes écoles mais aussi des industriels qui développent des innovations marquantes. Pour cette première édition, cette nouvelle distinction a été attribuée à trois chercheurs réputés : l\'économiste Esther Duflo, le roboticien François Pierrot et le physicien Mathias Fink

In [7]:
CORPUS = CorpusClient.from_folder(BS4CLIENT, "data/BULLETINS", limit=10)

In [8]:
len(CORPUS.documents)

10

In [9]:
CORPUS.segmente()

,token,filename
0,pu,67068.htm
1,linnovation,67068.htm
2,ajoute,67068.htm
3,fondamentaux,67068.htm
4,la,67068.htm
...,...,...
2530,limportance,67391.htm
2531,membranaire,67391.htm
2532,pays,67391.htm
2533,malgré,67391.htm
